<a href="https://colab.research.google.com/github/JimKing100/Jestimate/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import libraries
import pandas as pd
import numpy as np

In [4]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/SF-SFR-Sales-Final2d.csv')

# Rename subdistr_desc to neighborhood
df = df.rename(columns={'subdist_desc': 'neighborhood'})

# Check the data
print(df.shape)
df.head(5)

(23711, 40)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_nox,street_suffix,zip,area,district_no,district_desc,subdist_no,neighborhood,on_market_data,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,41347,4,1095000,41355,1260000,8,3.5,4,T,Per Tax Records,2691,NaN,0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN"
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,NaN,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,42341,144,1250000,42485,1075000,9,3.0,4,T,Per Tax Records,2437,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,NaN,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,42873,36,1395000,42964,1525000,9,5.0,5,D,Per Architect,2597,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,NaN,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,40356,42,725000,40410,715000,5,1.0,2,T,Per Tax Records,1312,NaN,0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK"
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,NaN,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,41067,24,1595000,41103,1595000,8,4.0,4,T,Per Tax Records,3307,NaN,0,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK"


In [5]:
# Check for nulls
df.isnull().sum()

longitude            469
latitude             469
elevation            469
full_address           0
city                   0
state                  0
street_no              0
street_name            0
street_nox         23460
street_suffix       1613
zip                    0
area                   0
district_no            0
district_desc          0
subdist_no             0
neighborhood           0
on_market_data         0
cdom                   0
orig_list_price        0
sale_date              0
sale_price             0
rooms                  0
baths                  0
beds                   0
sf_source              0
sf_source_decs         0
sf                     0
lot_acres          13379
lot_sf                 0
year_built             0
zoning             10880
lot_desc               0
drive_side             0
parking                0
park_leased            0
num_parking            0
shopping               0
transportation         0
type                   0
views                  0


In [9]:
# Check for zeros
(df == 0).sum()

longitude              0
latitude               0
elevation              0
full_address           0
city                   0
state                  0
street_no              0
street_name            0
street_nox             0
street_suffix          0
zip                    0
area                   0
district_no            0
district_desc          0
subdist_no             0
neighborhood           0
on_market_data         0
cdom                 806
orig_list_price        0
sale_date              0
sale_price             0
rooms               7251
baths                 26
beds                 141
sf_source              0
sf_source_decs         0
sf                  3781
lot_acres              1
lot_sf             13404
year_built          1409
zoning                 0
lot_desc               0
drive_side             0
parking                0
park_leased            0
num_parking         1412
shopping               0
transportation         0
type                   0
views                  0


In [11]:
# Baseline of sale_price
df['sale_price'].mean()

1371454.081312471

In [0]:
# Train, test split on date of 10/1/2018
df['game_date'] = pd.to_datetime(df['game_date'], infer_datetime_format=True)
cutoff = pd.to_datetime('2018-10-01')
train = df[df['game_date'] < cutoff]
test  = df[df['game_date'] >= cutoff]

print(train.shape)
print(test.shape)